In [1]:
import json
import operator
import copy

In [2]:
with open('./Workspace/result/BlogWithType.json', 'r') as f:
    blog = json.load(f)

In [3]:
with open('./Workspace/result/RestaurantWithType.json', 'r') as f:
    restaurant = json.load(f)

In [5]:
for b in blog:
    name = restaurant[blog[b]['restaurant_id']]['name']
    region = restaurant[blog[b]['restaurant_id']]['region']
    for psubject in blog[b]['positiveSubject']:
        psubject.append(copy.deepcopy(name))
        psubject.append(copy.deepcopy(region))
    for nsubject in blog[b]['negativeSubject']:
        nsubject.append(copy.deepcopy(name))
        nsubject.append(copy.deepcopy(region))

In [6]:
print blog.values()[52].keys()

[u'negativeSubject', u'blog_type', u'title', u'url', u'timestamp', u'is_paid', u'recommend_cnt', u'restaurant_type', u'browse_cnt', u'favorite_cnt', u'score', u'share_cnt', u'date', u'article', u'author_id', u'_id', u'restaurant_id', u'positiveSubject']


In [7]:
with open('./Workspace/result/BlogWithSubjectAndName.json' ,'w') as f:
    json.dump(blog, f)

In [33]:
with open('./Workspace/data/sentimentResult_20160701.json', 'r') as f:
    simplesubject= json.load(f)

In [35]:
food = []
for ele in simplesubject:
    food.extend(ele['positiveSubject'])
    food.extend(ele['negativeSubject'])

In [44]:
getter = operator.itemgetter(0)
foodlist = list(set(map(getter, food)))
print len(foodlist)

14543


In [45]:
from gensim import models
model = models.Word2Vec.load('./Workspace/Word2Vec_sz_200_mc_1_sg_by_sentence_big')

In [48]:
import time
vec = []
start = time.time()
for ele in foodlist:
    try:
        smvec = model[ele]
        for mos in model.most_similar(ele, topn = 10):
            smvec += model[mos[0]]
        smvec /= 11
        vec.append(smvec)
    except:
        pass
print time.time()-start

611.051765919


In [49]:
print len(vec)

14513


In [50]:
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_distances
def new_euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False):
    return cosine_distances(X,Y)
from sklearn.cluster import k_means_
k_means_.euclidean_distances = new_euclidean_distances

In [52]:
from sklearn.cluster import KMeans
kmeans_clustering = KMeans(n_clusters = 150)
idx = kmeans_clustering.fit_predict(vec)
r = dict(zip(foodlist, idx))
tempD = {}
for ele in r:
    if str(r[ele]) not in tempD:
        tempD[str(r[ele])] = [ele]
    else:
        tempD[str(r[ele])].append(ele)
with open('./Workspace/result/food_Kmean_150.json', 'w') as f:
    json.dump(tempD, f)